In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Bidirectional


from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [2]:
dataset = pd.read_csv(r'/content/drive/My Drive/Deep Learning/ECG/ECG-Dataset/ECG.csv')

In [3]:
dataset_500_signal = dataset[0:62500]
print(len(dataset_500_signal))

62500


In [4]:
dataset_500_signal.shape

(62500, 3)

In [5]:
X = dataset_500_signal.iloc[:, 1:-1]
Y = dataset_500_signal.iloc[:, -1]

In [6]:
# print("X= ", X.head())
# print("Y= ", Y.head())
print("X shape: ", X.shape)
print("Y shape: ", Y.shape)


X shape:  (62500, 1)
Y shape:  (62500,)


In [7]:
from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X = sc_X.fit_transform(X)
Y = sc_y.fit_transform(np.array(Y).reshape(-1,1))

In [8]:
X =np.array(X).reshape(62500, 1, 1)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [10]:
  # create model
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(1, 1)))
model.add(LSTM(110, activation='relu', return_sequences=True))
model.add(LSTM(50, activation='relu'))
model.add(Dense(4, activation='softmax'))
	# Compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 128)            66560     
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 110)            105160    
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                32200     
_________________________________________________________________
dense (Dense)                (None, 4)                 204       
Total params: 204,124
Trainable params: 204,124
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
model.fit(X_train, y_train, epochs=100, batch_size=5)

Epoch 1/100
10000/10000 [==============================] - 39s 4ms/step - loss: 0.5310 - accuracy: 0.5200
Epoch 2/100
10000/10000 [==============================] - 39s 4ms/step - loss: 0.4748 - accuracy: 0.5725
Epoch 3/100
10000/10000 [==============================] - 39s 4ms/step - loss: 0.4722 - accuracy: 0.5743
Epoch 4/100
10000/10000 [==============================] - 39s 4ms/step - loss: 0.4715 - accuracy: 0.5732
Epoch 5/100
10000/10000 [==============================] - 39s 4ms/step - loss: 0.4702 - accuracy: 0.5735
Epoch 6/100
10000/10000 [==============================] - 39s 4ms/step - loss: 0.4659 - accuracy: 0.5742
Epoch 7/100
10000/10000 [==============================] - 41s 4ms/step - loss: 0.4637 - accuracy: 0.5741
Epoch 8/100
10000/10000 [==============================] - 41s 4ms/step - loss: 0.4645 - accuracy: 0.5734
Epoch 9/100
10000/10000 [==============================] - 41s 4ms/step - loss: 0.4628 - accuracy: 0.5749
Epoch 10/100
10000/10000 [====================

In [12]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 58.06%


# **Cross Vlaidation**

In [13]:
dataset = pd.read_csv(r'/content/drive/My Drive/Deep Learning/ECG/ECG-Dataset/ECG.csv')
dataset_500_signal = dataset[0:62500]

X = dataset_500_signal.iloc[:, 1:-1]
Y = dataset_500_signal.iloc[:, -1]

from sklearn.preprocessing import MinMaxScaler
sc_X = MinMaxScaler()
sc_y = MinMaxScaler()
X = sc_X.fit_transform(X)
Y = sc_y.fit_transform(np.array(Y).reshape(-1,1))

X =np.array(X).reshape(62500, 1, 1)

In [14]:

# define baseline model
def baseline_model():
	# create model
  model = Sequential()
  model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(1, 1)))
  model.add(LSTM(110, activation='relu', return_sequences=True))
  model.add(LSTM(50, activation='relu'))
  model.add(Dense(4, activation='softmax'))
  model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [ ]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=100, batch_size=5, verbose=0)
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, Y, cv=kfold)
print("Accuracy: %.2f%%" % (results.mean()*100))
print("Starndard daviation: %.2f%%" % (results.std()*100))